In [1]:
import codecs
import os
from gensim import corpora, models, similarities
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from collections import defaultdict
from collections import Iterable
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
import re


In [2]:
brown_ic = wordnet_ic.ic('ic-brown.dat')

In [3]:
def make_syn(temp):
    for i in y:
        temp = wn.synsets(i)
        return temp

In [4]:
def measure_sim(list1):
    z = []
    for i in range(len(list1)):
        for j in range(i+1, len(list1)):
            z.append(wn.lin_similarity(list1[i], list1[j], brown_ic, verbose=False))
    if len(z) <2:
        return
    else:
        return (sum(z)/len(z))

In [5]:
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:        
             yield item

In [6]:
def lda_process(raw):
    our_stops = {'elizabeth','interprétation', 'clémence',
'Gazette', 'sur', 'loi', 'où', 'majesty', 'm', 'ance', 'q',
'xxvii', 'act', 'section', 'subsection', 'replaced', 'i','ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix','x',
'xi','xii', 'xiii','xix','xx','xxi','xxii','xxiii',
'code', 'part', 'revoked', 'subparagraphs', 'subparagraph','les', 'militaire', 'sans'}
    
    documents = word_tokenize(raw)
    sw = set(stopwords.words('english')) | our_stops
    texts = [w for w in documents if w.lower() not in sw and w.isalpha() and len(w)>2 ]
    frequency = defaultdict(int)
    fd = FreqDist(texts)
    dictionary = corpora.Dictionary([fd])
    corpus = [dictionary.doc2bow([text]) for text in texts]
    lda_out = models.LdaModel(corpus)
    corpus_lda = lda_out[corpus]
    lda = models.LdaModel(corpus_lda, id2word=dictionary, num_topics=5, passes=12)
    topic_list = lda.show_topics(num_topics = 5, num_words = 5, log=False, formatted=True)
    m = []
    for t in topic_list:
        m.append(re.findall(r'\*(\w+)', str(t)))
        
    y = list(flatten(m))
    noun = []
    
    for i in range(0,len(y)):
        temp = wn.synsets(y[i])
        if len(temp)==0:
            continue
        elif temp[0].pos()=='n':
            noun.append(temp[0])
            
    return measure_sim(noun)

In [7]:
for i in os.listdir(os.getcwd()):
    if i.endswith(".txt"):
        with codecs.open(i, 'r', encoding='utf-8') as f:
            raw = f.read()
            if len(raw) > 17000:
                with open('results.csv', 'a') as g:
                    #print(i, lda_process(raw), len(raw), sep=',', file=g)
                    g.writelines(str(i) + ',' + str(lda_process(raw)) + ',' + str(len(raw)) + '\n')
            else:
                with open('results.csv', 'a') as g:
                    g.writelines(str(i)+ ',' + '1,' + str(len(raw)) + '\n')
    
    